# wav2vec 2.0 learning_rate ablation

## Placeholders introduced
- `PATH_TO_DATASET` — where to place your dataset (e.g., data/ )
- `PATH_TO_OUTPUT_DIR` -path_to_output
- `PATH_TO_REPO` -path_to_base_repo
- `PARAMS` -different parameter values

In [ ]:
!pip install datasets soundfile librosa evaluate jiwer kaggle accelerate
!pip install pandas
!pip install huggingface_hub
!pip install wandb
!pip install transformers
!pip uninstall torch torchvision torchaudio -y
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu118

In [ ]:
import importlib.util
reqs=['transformers', 'datasets', 'soundfile', 'librosa', 'evaluate', 'jiwer', 'accelerate','pandas','wandb']
def check_installed(packages):
    for pkg in packages:
        try:
            importlib.import_module(pkg)
            print(f"{pkg} is installed")
        except ImportError:
            print(f"{pkg} is NOT installed")
check_installed(reqs)

In [ ]:
import torch, transformers
print(f"PyTorch: {torch.__version__}")  
print(f"Transformers: {transformers.__version__}")

# DTensor check
try:
    from torch.distributed.tensor import DTensor
    print(" DTensor available")
except ImportError:
    raise RuntimeError(" DTensor not found - upgrade PyTorch!")

In [ ]:
import os, json, numpy as np, pandas as pd
from pathlib import Path
from datasets import Dataset, Audio
from transformers import (
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    Wav2Vec2CTCTokenizer,
    TrainingArguments,
    Trainer
)
import torch
from dataclasses import dataclass
from typing import List, Dict, Union
import evaluate 
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import torch
from transformers import AutoProcessor, AutoModelForCTC
import wandb
from datetime import datetime
import re
from huggingface_hub import login

In [ ]:
json_path = "PATH_TO_DATASET/processed_json/combined_dataset.json"
AUDIO_BASE_PATH = "PATH_TO_DATASET/extracted/audio_files"
SAMPLING_RATE = 16000
print(f"Loading dataset from: {json_path}")
dataset = load_dataset("json", data_files=json_path, split="train")
print(f"Loaded dataset with {len(dataset)} samples")

print("Normalizing absolute paths to real audio files...")

def normalize_audio_path(example):
    relative_filename = example["audio"].split("clips/")[-1]
    example["audio"] = os.path.join(AUDIO_BASE_PATH, "clips", relative_filename)
    return example

dataset = dataset.map(normalize_audio_path)

data = [{"audio": x["audio"], "sentence": x["transcription"]} for x in dataset]
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=SAMPLING_RATE))
val_dataset = val_dataset.cast_column("audio", Audio(sampling_rate=SAMPLING_RATE))
print("Audio decoding complete!")
print("Sample entry:", train_dataset[0])

In [ ]:
print(f"Train set size: {len(train_dataset)}")
print(f" Validation set size: {len(val_dataset)}")

In [ ]:
telugu_special_unwanted_characters = [
    'ఁ', 'ౄ', 'ౢ', 'ౣ', 'ౠ', 'ఽ',
    '౦', '౧', '౨', '౩', '౪', '౫', '౬', '౭', '౮', '౯',
    'ఀ', 'ౘ', 'ౙ', 'ౚ', '౷',
    '‘', '’', '“', '”', '%', '.', ';', '-', ',', '/', '\\', '_', '&',
    'G', 'P', 'S', 'e', 'l', 'n', 'r', 't', '\u200c', '\n'
]

chars_to_remove_regex = f"[{re.escape(''.join(telugu_special_unwanted_characters))}]"
def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"])
    return batch

train_dataset = train_dataset.map(remove_special_characters)
val_dataset   = val_dataset.map(remove_special_characters)

print("Special characters removed from 'sentence' column.")

In [ ]:
base_repo_name = "PATH_TO_REPO"

processor = AutoProcessor.from_pretrained(base_repo_name)
model = AutoModelForCTC.from_pretrained(base_repo_name).to("cuda")

In [ ]:
model.freeze_feature_encoder()

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch
    
train_dataset = train_dataset.map(
    prepare_dataset,
    remove_columns=train_dataset.column_names,
    num_proc=4
)

val_dataset = val_dataset.map(
    prepare_dataset,
    remove_columns=val_dataset.column_names,
    num_proc=4
)

print("Datasets prepared and tokenized.")

In [ ]:
#prpeare data collator

from dataclasses import dataclass
from typing import Dict, List, Union
import torch
from transformers import Wav2Vec2Processor

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": f["input_values"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt"
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt"
            )

        
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels
        return batch

data_collator = DataCollatorCTCWithPadding(
    processor=processor,
    padding=True  
)
print("Data collator ready.")

In [ ]:
login(token="******") #insert token here

In [ ]:
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def compute_metrics(pred):
    pred_ids = np.argmax(pred.predictions, axis=-1)
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, group_tokens=False)
    return {
        "wer": wer_metric.compute(predictions=pred_str, references=label_str),
        "cer": cer_metric.compute(predictions=pred_str, references=label_str),
    }


In [ ]:
PARAMS={'EPOCHS':15, #Enter required epoch value
         'HIDDEN_DROPOUT': 0.1 #Enter required hidden dropout value
         }

In [ ]:
wandb.login()

In [ ]:
def train_with_lr(learning_rate, run_id):
    print(f"Training with learning rate: {learning_rate}")
    model = AutoModelForCTC.from_pretrained(base_repo_name).to("cuda")
    model.freeze_feature_encoder()
    output_dir = f"PATH_TO_OUTPUT_DIR/lr_ablation_{run_id}"
    repo_name = f"PATH_TO_REPO/lr_ablation_{run_id}"
    PARAMS = {
        "epochs": PARAMS['EPOCHS'],
        "batch_size": 8,
        "hidden_dropout": PARAMS['HIDDEN DROPOUT'],
        "sampling_rate": 16000
    }
    wandb.init(
    project="telugu-asr-wav2vec_ablation", 
    name=f"lr_ablation_{learning_rate}", 
    config={
        "learning_rate": learning_rate,
        "freeze_feature_encoder": True,
        "base_model": "finetunedxlsr-300m"
    }
    )
    training_args = TrainingArguments(
        output_dir=output_dir,
        group_by_length=True,
        per_device_train_batch_size=PARAMS["batch_size"],
        gradient_accumulation_steps=2,
        eval_strategy="steps",
        num_train_epochs=PARAMS["epochs"],
        gradient_checkpointing=True,
        fp16=True,  
        save_steps=200,
        eval_steps=200,
        logging_steps=50,
        learning_rate=learning_rate,
        warmup_ratio=0.1,
        save_total_limit=2,
        report_to="wandb",
        run_name=f"lr_ablation_{learning_rate}",
        push_to_hub=True,
        hub_model_id=repo_name,
        logging_dir=f"{output_dir}/logs"
    )
    
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=processor,
        data_collator=data_collator, 
        compute_metrics=compute_metrics
    )

    trainer.train()
    trainer.push_to_hub(commit_message=f"Trained with learning rate {learning_rate}")
    wandb.finish()

In [ ]:
train_with_lr(learning_rate=1e-4, run_id="1e-4")
train_with_lr(learning_rate=5e-5, run_id="5e-5")

In [ ]:
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def evaluate_model(repo_name: str, test_dataset):
    print(f"\nEvaluating: {repo_name}")
    processor = AutoProcessor.from_pretrained(repo_name)
    model = AutoModelForCTC.from_pretrained(repo_name).to("cuda")
    model.eval()

    def map_to_prediction(batch):
        with torch.no_grad():
            input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
            logits = model(input_values).logits
            pred_ids = torch.argmax(logits, dim=-1)
            batch["pred_str"] = processor.batch_decode(pred_ids)[0]
            batch["text"] = processor.decode(batch["labels"], group_tokens=False)
        return batch

    results = test_dataset.map(map_to_prediction, remove_columns=test_dataset.column_names)

    wer = wer_metric.compute(predictions=results["pred_str"], references=results["text"])
    cer = cer_metric.compute(predictions=results["pred_str"], references=results["text"])
    
    print(f"WER for {repo_name}: {wer:.4f}")
    print(f"CER for {repo_name}: {cer:.4f}")
    
    return wer, cer


In [ ]:
repo_names = [
    "PATH_TO_NEW_REPO1",
    "PATH_TO_NEW_REPO2"
]

results_dict = {}
for repo in repo_names:
    wer, cer = evaluate_model(repo, val_dataset)
    results_dict[repo] = {"WER": wer, "CER": cer}
for k,v in results_dict.items():
    print(f"output of {k}:\n{v}")